In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import keras
import keras_nlp

In [ ]:
import json
import csv
data = []
this = True

template = "Instruction:\n{instruction}\n\nResponse:\n{output}"
with open('/kaggle/input/finance-alpaca/finance_alpaca.json') as wrapper:
    for file in wrapper:
        counter = 0
        features = json.loads(file)
        for line in features:
            counter += 1
            data.append(template.format(**line))
            if counter >= 1000:
                break

with open ('/kaggle/input/financial-q-and-a-10k/Financial-QA-10k.csv') as file_obj_0:
    reader_obj_0 = csv.reader(file_obj_0)
    begin = True
    counter = 0
    for line in reader_obj_0:
        if begin:
            begin = False 
            continue
        counter += 1
        format_dict = {}
        format_dict['instruction'] = line[0]
        format_dict['output'] = line[1]
        if this:
                this = False
                print(format_dict)
        data.append(template.format(**format_dict))
        if counter >= 1000:
            break

with open ('/kaggle/input/investopedia-investment-qa/train.csv') as file_obj_1:
    reader_obj_1 = csv.reader(file_obj_1)
    begin = True
    counter = 0
    for line in reader_obj_1:
        if begin:
            begin = False
            continue
        counter += 1
        format_dict = {}
        format_dict['instruction'] = line[4]
        format_dict['output'] = line[5]
        data.append(template.format(**format_dict))
        if counter >= 1000:
            break
        

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
gemma_lm.summary()

In [ ]:
prompt = template.format(
    instruction="What should I look for when applying for a credit card as a college student?",
    output="",
)
print(gemma_lm.generate(prompt, max_length=1024))

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

In [ ]:
prompt = template.format(
    instruction="What should I look for when applying for a credit card as a college student?",
    output="",
)
print(gemma_lm.generate(prompt, max_length=1000))

In [ ]:
prompt = template.format(
    instruction="As a college student, how should I choose what investments I should make, such as in stocks and bonds?",
    output="",
)
print(gemma_lm.generate(prompt, max_length=1000))

In [ ]:
prompt = template.format(
    instruction="Explain the black-scholes model to me.",
    output="",
)
print(gemma_lm.generate(prompt, max_length=1000))

In [ ]:
prompt = template.format(
    instruction="Explain the black-scholes model to a college student who isn't mathematically inclined.",
    output="",
)
print(gemma_lm.generate(prompt, max_length=1000))